# Zephyr - 
## A cool breeze nudging those who can help toward those in need

##### Note: this notebook is built out of order for ease of presentation!

- To run it yourself start at the "CODE" section and run all the cells from that point down.
- Then run the presentation from the top.

##### Note: you will need to provide your own keys in the hidden "Credentials" cell



<img src = https://dl.dropboxusercontent.com/s/nrczftpxqz504yr/ArchitectureRev1.png width = 1200>

In [2]:
'''Here is a pseudocode example of our application's primary function:'''

def main():
    ben = "benaffleck"
    tweet = get_tweet(ben)
    weather = get_weather(tweet['location'])
    gloom = risk_score(tweet['text'], weather)
    
    if gloom > 4:
        matt = find_friend(ben)
        get_help(matt)
        send_help(ben, matt)

# Live Demo

In [89]:
'''
Here are some (very) simplified tweets from Ben Affleck and Matt Damon 
on a recent (fictional) day in Seattle.

(tweet text taken from real, live tweets the day of the competition 
. . . except for the one where Ben says he feels sad)
'''

Ben1 = {'tweet': getTweets("benaffleck")[0]['text'],
      'location':"111 S Jackson St, Seattle, WA 98104",
      'date':"20170326"
      }

Ben2 = {'tweet': 'I feel sad',
      'location':"111 S Jackson St, Seattle, WA 98104",
      'date':"20170326"
      }

Matt = {'tweet': getTweets("mattdamon_")[0]['text'],
      'location':"111 S Jackson St, Seattle, WA 98104",
      'date':"20170326"
      }

print('Ben tweets in the morning: {}\n\nMatt tweets: {}\n\nThen Ben tweets: {}'\
      .format(Ben1['tweet'], Matt['tweet'], Ben2['tweet']))

Ben tweets in the morning: RT @justiceleaguewb: Justice for all. #JusticeLeague in theaters November 17. https://t.co/p01PMDzAsM

Matt tweets: RT @RealHughJackman: The race to #operationchange is on. Follow clues @operationchange and you may win a trip around the world. Game on!

Then Ben tweets: I feel sad


In [90]:
'''
Our app calculates a running risk-score that uses 
tone analysis combined with current weather data 
to determine whether a user is likely to be depressed
'''

Ben1_risk = find_risk(Ben1['tweet'])
Ben2_risk = find_risk(Ben2['tweet'])
Matt_risk = find_risk(Matt['tweet'])

mat_weather = historical_weather(Matt['location'], Matt['date'])
mat_gloom = historical_gloom(matt_weather)

ben_weather = historical_weather(Ben1['location'], Ben1['date'])
ben_gloom = historical_gloom(ben_weather)

matt_score = combined_risk(mat_gloom, matt_risk)
ben1_score = combined_risk(ben_gloom, Ben1_risk)
ben2_score = combined_risk(ben_gloom, Ben2_risk)

print('Matt: {}\nBen in the morning: {}\nBen later that day: {}'.format(matt_score, ben1_score, ben2_score))

Matt: 1
Ben in the morning: 4
Ben later that day: 6


## Alert: Ben's gloom score has risen above 4.
## Ben Affleck is sad! Send help!
![Sad Ben](https://github.com/Coord/zephyr/blob/master/resources/sad-ben-affleck.gif?raw=true”)

In [ ]:
'''
See code section to see how promotion is actually sent
'''

sendPromotion('Matt Damon', "<mattdamon's email>", message)

![Email](https://cdn.rawgit.com/Coord/zephyr/b8f3018b/images/email.png "Email")

## Thanks to project Zephyr, Matt recieves a promotion to encourage him to take Ben to the gym - a local and weather appropriate activity, specially chosen to lift Ben's mood!

In [3]:
'''End Presentation'''

'End Presentation'

# Code

In [19]:
!pip install --user twython

  Using cached twython-3.4.0.tar.gz
  Using cached requests_oauthlib-0.8.0-py2.py3-none-any.whl
  Using cached oauthlib-2.0.2.tar.gz
  Running setup.py bdist_wheel for twython ... - \ done
  Stored in directory: /gpfs/fs01/user/sb19-bf3be4d4b96e6f-9552eb6e93f3/.cache/pip/wheels/48/e9/f5/a4c968725948c73f71df51a3c6759425358c1eda2dcf2031f4
  Running setup.py bdist_wheel for oauthlib ... - \ done
  Stored in directory: /gpfs/fs01/user/sb19-bf3be4d4b96e6f-9552eb6e93f3/.cache/pip/wheels/84/98/7a/fba7268f61097bea6081cbe5480bc439b38975748ea7684fd5
Successfully built twython oauthlib


In [79]:
import numpy as np
import requests
import os
from os.path import join, dirname
import datetime

from IPython.display import Image 
from IPython.core.display import HTML

import json
from watson_developer_cloud import ToneAnalyzerV3
from twython import Twython

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [17]:
# @hidden_cell
# Credentials *** HIDE THIS CELL ***
WEATHER_USERNAME = "<your bluemix Weather Company Data username>"
WEATHER_PASSWORD = "<your bluemix Weather Company Data password>"
API_KEY = "<your weather.com api key>"
TONE_USERNAME="<your bluemix tone analyzer username>"
TONE_PASSWORD="<your bluemix tone analyzer password>"
CONSUMER_KEY = "<your twitter consumer key>"
CONSUMER_SECRET = "<your twitter consumer secret key>"
ACCESS_KEY = "<your twitter access key>"
ACCESS_SECRET = "<your twitter access secret key>"
GMAIL_USERNAME = "<your gmail username>"
GMAIL_PASSWORD = "<your gmail password>"

# Promotional Email Messaging functions

In [ ]:
## Load promotion dataset from Object Storage


In [3]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_7118af08c7254fa09a49d92b625a83e3(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_008b9f8e4271482355b49de00d49ff1d409d4465','domain': {'id': 'c6c444b793c54f1bb15f25afb62bf1e5'},
            'password': 'Jh5&Jq^[]c}O}604'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

promotions = pd.read_csv(get_object_storage_file_with_credentials_7118af08c7254fa09a49d92b625a83e3('Weather', 'promotions.csv'))
promotions.head()


,weather,promotion,address
0,indoor,30% off tickets to Mesuem of Pop Culture,"325 5th Avenue N, Seattle WA 98109�"
1,indoor,$10 off of dinners for two at Wild Ginger Rest...,"11020 NE 6th St #90, Bellevue, WA 98004"
2,indoor,a free day pass to Gold's Gym,"2720 4th Ave #115, Seattle, WA 98121"
3,outdoor,30% off for tickets to the Wildwaves,"36201 Enchanted Pkwy S, Federal Way, WA 98003"
4,outdoor,a free guilded tours at the Arboretum,"2300 Arboretum Dr E, Seattle, WA 98112"


In [ ]:
## Generate promotion based on weather

In [5]:
import random

indoorPromotions = promotions[promotions.weather == "indoor"][["promotion"]].as_matrix()
outdoorPromotions = promotions[promotions.weather == "outdoor"][["promotion"]].as_matrix()

def getPromotionMessage(niceWeather, userInCrisis, userFriend):
    template = "Congratulations {}! You have recieved a {} to enjoy with {}! Click here to redeem!"
    promotions = indoorPromotions if niceWeather else outdoorPromotions
    promotion = random.choice(promotions)
    return template.format(userFriend, promotion[0], userInCrisis)

forecast = forecast_weather("111 S Jackson St, Seattle, WA 98104")
niceWeather = outdoor_bool(forecast)
userInCrisis = "Ben Affleck"
userFriend = "Matt Damon"

message = getPromotionMessage(niceWeather, userInCrisis, userFriend)
message

"Congratulations Matt Damon! You have recieved a $10 Fresh Bucks after you spend $20 at any Seattle Area Farmer's Market to enjoy with Ben Affleck! Click here to redeem!"

In [ ]:
## Send promotion out to friend

In [ ]:
import smtplib
from email.mime.text import MIMEText

def sendPromotion(name, email, message):
    sender = GMAIL_USERNAME # "zephyr.mail.alert@gmail.com"

    msg = MIMEText(message, 'plain')
    msg['From'] = sender
    msg['To'] = email
    msg['Subject'] = "Share this coupon with your friend: {}".format(name)
    mail = msg.as_string()
    
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender, GMAIL_PASSWORD)    
        server.sendmail(sender, email, mail)
        server.quit()
        print("Mail sent successfully!") 

    except Exception as exc:
        print("Failed to send mail")    
    
friendName = "Matt Damon"
friendEmail = "<mattdamon's email>"

sendPromotion(friendName, friendEmail, message)

# Weather functions

In [4]:
def historical_weather(address, end_date, units='e'):
    """
    Input: address, end date and units (strings)
    Output: historical weather data (json)
    Date format: 2017-03-25 -> '20170325'
    """
    # Get latude and longitude coordinates for given address
    payload = {'query': address,
               'locationType': 'address',
               'language': 'en-US'}
    r = requests.get('https://' +
                     WEATHER_USERNAME + ':' + WEATHER_PASSWORD +
                     '@twcservice.mybluemix.net/api/weather/v3/location/search',
                     params=payload)
    lat = r.json()['location']['latitude'][0]
    lon = r.json()['location']['longitude'][0]
    # Find start_date one week earlier
    end_dt = datetime.date(int(end_date[0:4]),
                           int(end_date[4:6]),
                           int(end_date[6:]))
    td = datetime.timedelta(days=7)
    start_dt = end_dt - td
    year = str(start_dt.year)
    month = str(start_dt.month)
    if len(month) == 1:
        month = '0' + month
    day = str(start_dt.day)
    if len(day) == 1:
        day = '0' + day
    start_date = (year + month + day)
    # Get 7 day forcast from latude and longitude coordinates
    payload = {'units': 'e',
               'startDate': start_date,
               'endDate': end_date}
    r = requests.get('http://api.weather.com/v1/geocode/' +
                     str(lat) + '/' + str(lon) +
                     '/observations/historical.json?apiKey=' +
                     API_KEY, params=payload)
    return r.json()


def forecast_weather(address, units='e'):
    """
    Input: address and units (strings)
    Output: 7 day weather forecast (json)
    Date format: 2017-03-25 -> '20170325'
    """
    # Get latude and longitude coordinates for given address
    payload = {'query': address, 'locationType': 'address', 'language': 'en-US'}
    r = requests.get('https://' +
                     WEATHER_USERNAME + ':' + WEATHER_PASSWORD +
                     '@twcservice.mybluemix.net/api/weather/v3/location/search',
                     params=payload)
    lat = r.json()['location']['latitude'][0]
    lon = r.json()['location']['longitude'][0]
    # Get 7 day forcast from latude and longitude coordinates
    payload = {'units': units}
    r = requests.get('https://' +
                     WEATHER_USERNAME + ':' + WEATHER_PASSWORD +
                     '@twcservice.mybluemix.net/api/weather/v1/geocode/' +
                     str(lat) + '/' + str(lon) +
                     '/forecast/daily/7day.json',
                     params=payload)
    return r.json()


def historical_gloom(hist_json):
    """
    Input: historical weather information (json)
    Output: bad weather score (float)
    """
    mean_clouds = []
    mean_precip = []
    # count instances of overcast skies and precipitation
    for ob in hist_json['observations']:
        if ob['clds'] == 'OVC':
            mean_clouds.append(1)
        else:
            mean_clouds.append(0)
        if type(ob['precip_hrly']) == float and ob['precip_hrly'] > 0:
            mean_precip.append(1)
        else:
            mean_precip.append(0)
    # find gloom score and scale from 0 to 1
    return (np.mean(mean_clouds) + (2 * np.mean(mean_precip))) / 3


def outdoor_bool(forecast_json):
    """
    Input: 7 day weather forecast (json)
    Output: outside activity bool
    """
    # get forecast for next day
    forecast = forecast_json['forecasts'][1]['day']
    # return bool for low prop precip and comfortable windchill
    return forecast['pop'] <= 0.2 and forecast['wc'] >= 40


# Tweet Tone functions

In [40]:
def find_risk(tweet, testing=False):
    
    tone_analyzer = ToneAnalyzerV3(username=TONE_USERNAME,
                                    password=TONE_PASSWORD,
                                    version='2016-05-19')

    # crisis vector is determined experimentally by analzing tweets
    # for sentiments that reflect when a person may be depressed
    # or otherwise at risk

    # ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness']
    crisis = np.array([0.02, 0.04, 0.01, 0, 0.93]).reshape(1, -1)

    json_response = tone_analyzer.tone(text=tweet)
    tones_json = json_response['document_tone']['tone_categories'][0]['tones']
    sentiment = np.array([d['score'] for d in tones_json]).reshape(1, -1)

    # Find cosine similarity between sentiment of current tweet
    # and crisis vector
    risk = cosine_similarity(crisis, sentiment)[0][0]
    risk = round(risk,4)
    if testing == True:
        emotion = 'Ang Disg Fear Joy Sad'
        sentiment = [round(float(s),4) for s in sentiment[0]]
        risk = 'emotion: {}\ncrisis:  {}\ntweet: {}\nrisk score: {}'\
                .format(emotion, crisis[0], sentiment, risk)

    return risk


# Combined Score

In [53]:
def combined_risk(weather_risk, tweet_risk):
    combined = np.mean([weather_risk, 2*tweet_risk])
    combined_score = int(round(combined * 5))
    if combined_score == 0:
        combined_score += 1
    return combined_score



# Get Live Tweets

In [21]:
def getTweets(name):
    twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET)

    # tweet extract method with the last list item as the max_id
    user_timeline = twitter.get_user_timeline(screen_name=name,
                                              count=10,
                                              include_retweets=False)
    return user_timeline
